<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-28 14:21:30
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  10.00 K
Current:  1.33 C
-------------------
Today PnL: -1.39 L (-1.04%)
Current PnL: -18.30 L (-12.75%)
CY Booked + Current PnL: -10.22 L (-7.12%)
-------------------
Total profit:  2.49 L
Total loss:  -20.80 L
-------------------
Total Booked + Current PnL: 16.76 L (14.38%)
Total Booked PnL: 35.06 L (30.08%)
Curr Year Booked PnL: 8.08 L (6.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 76.52 L (57.59%)
Deployed:  1.17 C
Current:  1.33 C
CAGR/XIRR %: 8.47%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-219.72,H-SC,11.50,86317.0,-8999.0,9037.0,-1.49,-9.44,10.47,0.04,161.0,-1.00,0.65,70.25,OX40N,NTT,MISC
39,HINDALCO,651.95,761.55,-19.90,H-LC,5.42,110184.0,5872.0,11668.0,-0.69,5.63,10.59,16.81,10.0,0.50,0.83,22.58,X5K,ATH,METALS
4,APOLLOHOSP,6904.43,8285.00,-19.18,H-LC,6.78,110138.0,6572.0,14142.0,-1.69,6.35,12.84,20.00,14.0,0.46,0.83,21.31,X40N,BTT,HEALTHCARE
17,BRITANNIA,5190.70,6446.05,12.50,H-LC,9.73,100953.0,7520.0,15072.0,0.26,8.05,14.93,24.18,35.0,0.50,0.76,22.58,XY25,ATH,FMCG
58,MEDANTA,1087.93,1486.00,21.96,H-SC,10.52,156217.0,30017.0,16153.0,1.78,23.79,10.34,36.59,154.0,1.86,1.18,39.28,XY24,NTT,HEALTHCARE


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,QUESS,296.83,986.00,-34.99,M-SC,42.27,64714.0,-292.0,151217.0,-2.22,-0.45,233.67,232.18,196.0,-0.00,0.49,0.00,XY24,NTT,MISC
53,KPIGREEN,497.21,731.64,23.10,H-SC,12.61,125950.0,653.0,58428.0,-3.86,0.52,46.39,47.15,143.0,0.01,0.95,57.57,MH,ATH,POWER
14,BERGEPAINT,561.33,680.00,-19.92,H-MC,11.14,226820.0,-519.0,48585.0,0.37,-0.23,21.42,21.14,108.0,-0.01,1.71,27.00,XY24,NTT,PAINTS
38,HEROMOTOCO,4311.81,5949.07,-6.67,H-MC,2.83,149905.0,-1008.0,58313.0,1.24,-0.67,38.90,37.97,89.0,-0.02,1.13,24.47,AR,ATH,AUTO


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,CIPLA,1492.70,1795.0,-18.42,H-LC,1.09,214542.0,10042.0,31366.0,2.19,4.91,14.62,20.25,7.0,0.32,1.62,14.07,X40N,BTT,PHARMA
56,LTIM,5564.16,7230.2,-5.66,H-LC,2.45,181476.0,-18834.0,78815.0,-1.08,-9.40,43.43,29.94,26.0,-0.24,1.37,25.37,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-8.98,H-LC,2.51,188994.0,-11512.0,60289.0,-1.52,-5.74,31.90,24.33,30.0,-0.19,1.42,8.98,X40N,BTT,FINANCE
40,HINDUNILVR,2413.81,2723.0,-12.98,H-LC,2.99,272664.0,2317.0,32311.0,0.79,0.86,11.85,12.81,18.0,0.07,2.05,14.03,XY25,NTT,FMCG
5,ASIANPAINT,2961.56,4250.0,-19.64,H-LC,3.52,159535.0,-41851.0,129463.0,0.42,-20.78,81.15,43.51,20.0,-0.32,1.20,11.05,X40,BTT,PAINTS


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,SHALBY,261.39,327.00,986.12,M-SC,9.71,147706.0,-33437.0,78905.0,2.69,-18.46,53.42,25.10,228.0,-0.42,1.11,18.37,XY24,NTT,HEALTHCARE
22,CIPLA,1492.70,1795.00,-18.42,H-LC,1.09,214542.0,10042.0,31366.0,2.19,4.91,14.62,20.25,7.0,0.32,1.62,14.07,X40N,BTT,PHARMA
58,MEDANTA,1087.93,1486.00,21.96,H-SC,10.52,156217.0,30017.0,16153.0,1.78,23.79,10.34,36.59,154.0,1.86,1.18,39.28,XY24,NTT,HEALTHCARE
26,DABUR,505.20,735.00,-6.46,H-MC,4.28,202527.0,5499.0,84130.0,1.55,2.79,41.54,45.49,96.0,0.07,1.53,16.20,XY24,BTT,FMCG
91,VBL,492.64,672.28,-14.22,H-LC,12.74,260392.0,-4648.0,101292.0,1.44,-1.75,38.90,36.46,2.0,-0.05,1.96,11.21,X40N,ATH,FMCG


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
67,SAIL,130.64,228.00,78.75,M-MC,13.79,214647.0,-10315.0,177964.0,-4.61,-4.59,82.91,74.53,193.0,-0.06,1.62,24.63,XY24,BTT,STEEL
16,BLUSPRING,226.45,94.95,NaN,NaN,5.54,19598.0,-29995.0,1195.0,-4.48,-60.48,6.10,-58.07,204.0,-25.10,0.15,31.43,XY24,ATH,MISC
55,LAOPALA,369.40,464.00,129.14,H-SC,6.45,72877.0,-27969.0,53798.0,-4.37,-27.73,73.82,25.61,135.0,-0.52,0.55,31.44,AR,NTT,CERAMICS
28,DIGITIDE,188.38,267.99,NaN,NaN,2.22,52472.0,11217.0,6218.0,-4.25,27.19,11.85,42.26,197.0,1.80,0.40,23.54,XY24,ATH,IT
79,TANLA,917.30,1963.11,-33.71,H-SC,6.65,162159.0,-77256.0,350215.0,-4.25,-32.27,215.97,114.01,130.0,-0.22,1.22,48.73,AR,ATH,IT


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-219.72,H-SC,11.50,86317.0,-8999.0,9037.0,-1.49,-9.44,10.47,0.04,161.0,-1.00,0.65,70.25,OX40N,NTT,MISC
78,SYMPHONY,1306.42,1306.0,-31.84,M-SC,9.75,145279.0,-25862.0,25802.0,-0.63,-15.11,17.76,-0.03,194.0,-1.00,1.09,5.13,OX40N,NTT,DURABLES
86,UJJIVANSFB,52.77,53.0,36.52,M-SC,22.46,118503.0,-23976.0,24601.0,1.32,-16.83,20.76,0.44,249.0,-0.97,0.89,39.87,OX40N,NTT,BANKS
85,TTKPRESTIG,769.29,770.0,-9.13,H-SC,12.01,82995.0,-17782.0,17877.0,0.44,-17.64,21.54,0.09,144.0,-0.99,0.63,8.00,OX40N,NTT,DURABLES
80,TATAELXSI,7332.28,7332.0,-19.50,H-MC,1.61,84455.0,-18197.0,18192.0,-0.49,-17.73,21.54,-0.00,100.0,-1.00,0.64,28.38,OX40N,NTT,IT


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.44,73295.0,38846.0,6120.0,-1.53,112.77,8.35,130.54,1.0,6.35,0.55,12.08,AR,ATH,ELECTRICAL
91,VBL,492.64,672.28,-14.22,H-LC,12.74,260392.0,-4648.0,101292.0,1.44,-1.75,38.90,36.46,2.0,-0.05,1.96,11.21,X40N,ATH,FMCG
82,TCS,3794.03,4998.00,-26.09,H-LC,12.39,240856.0,-55078.0,148994.0,-1.53,-18.61,61.86,31.73,3.0,-0.37,1.81,0.00,X40,BTT,IT
48,INFY,1461.46,2275.00,-15.65,H-LC,6.89,270129.0,8528.0,137090.0,-0.44,3.26,50.75,55.67,5.0,0.06,2.03,9.56,X40,BTT,IT
1,ABB,5551.76,7934.00,-39.48,H-LC,7.29,216060.0,-459.0,93360.0,-1.83,-0.21,43.21,42.91,6.0,-0.00,1.63,12.94,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,2.22,52472.0,11217.0,6218.0,-4.25,27.19,11.85,42.26,197.0,1.80,0.40,23.54,XY24,ATH,IT
58,MEDANTA,1087.93,1486.00,21.96,H-SC,10.52,156217.0,30017.0,16153.0,1.78,23.79,10.34,36.59,154.0,1.86,1.18,39.28,XY24,NTT,HEALTHCARE
32,ENRIN,1377.95,3176.70,NaN,NaN,2.44,73295.0,38846.0,6120.0,-1.53,112.77,8.35,130.54,1.0,6.35,0.55,12.08,AR,ATH,ELECTRICAL


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,KPIGREEN,497.21,731.64,23.10,H-SC,12.61,125950.0,653.0,58428.0,-3.86,0.52,46.39,47.15,143.0,0.01,0.95,57.57,MH,ATH,POWER
58,MEDANTA,1087.93,1486.00,21.96,H-SC,10.52,156217.0,30017.0,16153.0,1.78,23.79,10.34,36.59,154.0,1.86,1.18,39.28,XY24,NTT,HEALTHCARE
20,CAMS,3643.00,5226.82,0.53,H-SC,1.57,112960.0,10956.0,33391.0,-3.87,10.74,29.56,43.48,122.0,0.33,0.85,29.32,X40N,ATH,MISC
26,DABUR,505.20,735.00,-6.46,H-MC,4.28,202527.0,5499.0,84130.0,1.55,2.79,41.54,45.49,96.0,0.07,1.53,16.20,XY24,BTT,FMCG
15,BLUESTARCO,1646.70,2326.38,-7.57,H-SC,10.52,173430.0,8760.0,59209.0,-0.74,5.32,34.14,41.28,119.0,0.15,1.31,13.73,X40N,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3176.70,NaN,NaN,2.44,73295.0,38846.0,6120.0,-1.53,112.77,8.35,130.54,1.0,6.35,0.55,12.08,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1219.98,2270.00,-38.43,M-SC,8.84,103530.0,12032.0,66725.0,-1.33,13.15,64.45,86.07,205.0,0.18,0.78,49.77,XR,NTT,DURABLES
45,INDIAMART,2327.09,4850.92,-48.08,H-SC,12.83,135373.0,12037.0,121727.0,-0.25,9.76,89.92,108.45,118.0,0.10,1.02,34.29,AR,ATH,MISC
95,WIPRO,243.46,420.00,-9.67,M-LC,5.68,154907.0,3962.0,105492.0,-1.71,2.62,68.10,72.51,56.0,0.04,1.17,8.81,XR,NTT,IT
53,KPIGREEN,497.21,731.64,23.10,H-SC,12.61,125950.0,653.0,58428.0,-3.86,0.52,46.39,47.15,143.0,0.01,0.95,57.57,MH,ATH,POWER


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,94.95,NaN,NaN,5.54,19598.0,-29995.0,1195.0,-4.48,-60.48,6.10,-58.07,204.0,-25.10,0.15,31.43,XY24,ATH,MISC
74,SIS,477.00,477.00,2270.98,M-SC,5.72,63792.0,-17298.0,17300.0,-1.26,-21.33,27.12,0.00,238.0,-1.00,0.48,29.29,OX40N,NTT,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,20.80,66586.0,-46963.0,87015.0,-3.80,-41.36,130.68,35.27,267.0,-0.54,0.50,62.97,XR,NTT,HOTELS
92,VIPIND,488.80,489.00,-219.72,H-SC,11.50,86317.0,-8999.0,9037.0,-1.49,-9.44,10.47,0.04,161.0,-1.00,0.65,70.25,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,-0.47,H-SC,20.24,92665.0,-8298.0,102830.0,-1.83,-8.22,110.97,93.63,147.0,-0.08,0.70,38.02,SR,ATH,CHEMICALS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
28,DIGITIDE,188.38,267.99,NaN,NaN,2.22,52472.0,11217.0,6218.0,-4.25,27.19,11.85,42.26,197.0,1.80,0.40,23.54,XY24,ATH,IT
33,GREENPANEL,375.16,537.00,254.01,M-SC,9.86,158088.0,-24990.0,103974.0,1.08,-13.65,65.77,43.14,234.0,-0.24,1.19,48.70,XY24,NTT,MISC
58,MEDANTA,1087.93,1486.00,21.96,H-SC,10.52,156217.0,30017.0,16153.0,1.78,23.79,10.34,36.59,154.0,1.86,1.18,39.28,XY24,NTT,HEALTHCARE
16,BLUSPRING,226.45,94.95,NaN,NaN,5.54,19598.0,-29995.0,1195.0,-4.48,-60.48,6.10,-58.07,204.0,-25.10,0.15,31.43,XY24,ATH,MISC
72,SHALBY,261.39,327.00,986.12,M-SC,9.71,147706.0,-33437.0,78905.0,2.69,-18.46,53.42,25.10,228.0,-0.42,1.11,18.37,XY24,NTT,HEALTHCARE


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.46
1,25,41.19
2,50,70.17


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    41.70
LC    35.15
MC    22.08
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.91
X40N     14.63
X40      12.54
XY25     10.76
XR        9.12
OX40N     8.38
AR        8.22
X5K       2.25
MH        1.70
X200      1.37
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    26.90
H-SC    25.26
H-MC    19.10
M-SC    15.05
M-LC     7.20
M-MC     2.63
L-SC     1.39
L-LC     1.05
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.14,-4.25,39.13
IT,11.77,-16.46,73.85
FINANCE,8.30,-18.09,60.94
BANKS,7.58,-13.33,59.57
MISC,6.12,-18.50,73.84
PAINTS,5.62,-16.95,40.00
ELECTRICAL,5.40,-2.77,51.47
HEALTHCARE,4.66,2.39,23.54
AUTO,4.42,-15.32,65.50


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2748973.0,25
XR,1083739.0,13
AR,1054622.0,10
X40,786796.0,10
X40N,608148.0,12
XY25,508988.0,8
OX40N,359405.0,11
SR,190765.0,2
X5K,140955.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2467628.0,26
M-SC,1632548.0,20
H-LC,1292322.0,20
H-MC,1173087.0,15
M-LC,422822.0,5
M-MC,329469.0,2
L-SC,214116.0,3
L-MC,56596.0,1
L-LC,49493.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,957230.0,7
M-SC,XY24,776338.0,7
H-SC,AR,525740.0,3
H-LC,X40,508543.0,6
H-MC,XY24,452226.0,4
H-SC,XR,359713.0,4
M-MC,XY24,329469.0,2
H-LC,X40N,312804.0,6
M-SC,XR,289104.0,4


In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 28.0 seconds
